In [17]:
%load_ext autoreload
%autoreload 2
import litebird_sim as lbs
import numpy as np
import healpy as hp
import sbm
import logging
import toml
from pathlib import Path
logging.basicConfig(level=logging.WARNING);

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
CONFIG_PATH = Path.home() / ".config" / "litebird_imo"
CONFIG_FILE_PATH = CONFIG_PATH / "imo.toml"
tomlinfo = toml.load(CONFIG_FILE_PATH)
flatfile_location = tomlinfo["repositories"][0]["location"]

imo = lbs.Imo(flatfile_location=flatfile_location)

In [19]:
channel = sbm.channel_list[0]

config = sbm.Configlation(imo, channel)
config.mdim = 3
config.parallel = True
config.nside = 128

syst = sbm.Systematics()
syst.sigma_gain_t = 0.1
syst.sigma_gain_b = 0.1

syst.sigma_rho_t = np.deg2rad(1/60)
syst.sigma_rho_b = 0.

syst.sigma_chi_b = 0.
syst.sigma_chi_t = 0.


Here, we will take a `N` realisations for CMB, systematics variations and noise.


In [25]:
N = 2 # number of realizations
npix = hp.nside2npix(config.nside)
gain_maps = np.zeros([N,N,N,3,npix]) # [cmb_seed, syst_seed, noise_seed, stokes, npix]
pointing_maps = np.zeros([N,N,N,3,npix])
noise_maps = np.zeros([N,3,npix])
fg_models = ["pysm_dust_1", "pysm_synch_1"]

for i in range(N):
    Mbsparams = lbs.MbsParameters(
        make_cmb=True,
        cmb_r=0.001,
        make_fg=True,
        seed_cmb=i,
        fg_models=fg_models,
        gaussian_smooth=False,
        bandpass_int=False,
        nside=config.nside,
        units="uK_CMB",
        maps_in_ecliptic=False,
    )
    for j in range(N):
        syst.syst_seed = j
        for k in range(N):
            syst.noise_seed = k
            print("CMB seed: ", i, " Syst. seed: ", j, " Noise seed: ", k)
            gain_map, noise_map, input_maps = sbm.sim_diff_gain_per_ch(config, syst, Mbsparams)
            pointing_map, noise_map, input_maps = sbm.sim_diff_pointing_per_ch(config, syst, Mbsparams)
            gain_maps[i,j,k] = gain_map
            pointing_maps[i,j,k] = pointing_map
            if i == 0:
                noise_maps[j] = noise_map

CMB seed:  0  Syst. seed:  0  Noise seed:  0


Processing L1-040: 100%|██████████| 24/24 [00:05<00:00,  4.60it/s]


CMB seed:  0  Syst. seed:  0  Noise seed:  1


Processing L1-040: 100%|██████████| 24/24 [00:04<00:00,  5.48it/s]


CMB seed:  0  Syst. seed:  1  Noise seed:  0


...

Processing L1-040: 100%|██████████| 24/24 [00:04<00:00,  5.65it/s]


CMB seed:  1  Syst. seed:  1  Noise seed:  0


Processing L1-040: 100%|██████████| 24/24 [00:05<00:00,  4.76it/s]


CMB seed:  1  Syst. seed:  1  Noise seed:  1


Processing L1-040: 100%|██████████| 24/24 [00:04<00:00,  5.43it/s]


In [30]:
noise_map

array([[ 6.73777638, -2.67859127, -2.13930331, ..., -4.99552282,
         2.74178377,  2.344675  ],
       [-7.27326591, -8.92202832, -9.01600881, ...,  3.79908528,
        -4.35283216, -4.35176604],
       [-0.69612227, -1.23294734,  6.26132003, ..., -0.12677082,
         2.62173038,  2.91296621]])